In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import joblib
import pandas as pd
import numpy as np
import faiss
sys.path.append('../')

os.environ['TRANSFORMERS_CACHE'] = '/data/users/kartik/hf_cache/'
os.environ['CUDA_VISIBLE_DEVICES']='0'

from prod_code.get_predictions import GetPredictions



2022-08-01 11:48:19.035818: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Seed set to 42


In [5]:
from utils.faiss_utils import create_and_store_index, get_top_n_accuracy
from trainer.params import get_roberta_params, get_xlm_params

params = get_xlm_params()
create_and_store_index(
    params['emb_type'], params['knn_measure']
    )


Loading Data for Indexing...
Creating FAISS Index...
Data len before unique_mapping: 7036
Data len after unique_mapping: 3996
Index shape (3996, 300)
FAISS Index Creation Done...


In [2]:

load_path='/data/users/kartik/Dictionary_and_Word_Embeddings/checkpoints/xlm-roberta-large_11'
getpred = GetPredictions(save_checkpoint_path=load_path, model_type='multilingual') #mono/multi


Preparing Model and Search Attributes...

Loading Tokenizer: xlm-roberta-large ...
Loading from : /data/users/kartik/Dictionary_and_Word_Embeddings/checkpoints/xlm-roberta-large_11/model.pt

Using device cuda:0 for training...


Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading Checkpoint ...
Setting Models state to checkpoint...
Model state set.
Index dict len: 3996


In [3]:
val = joblib.load('../data/clean_data/val.joblib')
ranks, nearest = getpred.get_predictions(val, NEAREST_NEIGHBOUR_NUM = 1000, pred_batch = 64) #

  0%|          | 0/55 [00:00<?, ?it/s]

Average Rank @1000 : 222.16752429959976
Median Rank @1000 : 37.0
Rank Variance @1000 : 338.5179719497607
Top 1 accuracy is 0.155
Top 2 accuracy is 0.216
Top 5 accuracy is 0.297
Top 10 accuracy is 0.366
Top 50 accuracy is 0.541
Top 100 accuracy is 0.628


In [4]:
val

,word,gloss,fasttext_aligned
0,AIS,", as an element of the Global Marine Distress ...","[-0.0867, 0.0185, -0.0289, 0.1101, -0.0047, 0...."
1,Addisonian,Of or relating to,"[0.1067, -0.0507, -0.0493, -0.0121, -0.0104, -..."
2,Asturian,Of or pertaining to the Asturian language .,"[0.025, 0.0495, 0.0007, -0.0087, -0.026, 0.049..."
3,Asturian,Of or pertaining to the region of Asturias in ...,"[0.025, 0.0495, 0.0007, -0.0087, -0.026, 0.049..."
4,BM,or,"[-0.0328, -0.0139, -0.0483, 0.0942, 0.0872, 0...."
...,...,...,...
3493,zircon,"A crystal of zircon , sometimes used as a fals...","[0.0267, -0.0014, -0.0297, -0.012, -0.0456, 0...."
3494,zircon,"A mineral occurring in tetragonal crystals , u...","[0.0267, -0.0014, -0.0297, -0.012, -0.0456, 0...."
3495,zoo,"Any place that is wild , crowded , or chaotic .","[-0.0047, -0.0617, 0.0026, 0.0046, -0.0421, 0...."
3496,zoonotic,Of or relating to zoonosis .,"[-0.044, -0.0255, -0.0769, 0.0777, -0.0504, -0..."


In [3]:
val = joblib.load('../data/clean_data/val.joblib')
# val = joblib.load('../data/clean_data/de_wiki_muse.joblib')
# ranks, nearest = getpred.get_predictions(val, NEAREST_NEIGHBOUR_NUM = 1000, pred_batch = 128) #

  0%|          | 0/887 [00:00<?, ?it/s]

Average Rank @1000 : 493.68910301578353
Median Rank @1000 : 348.0
Rank Variance @1000 : 446.60213713164956
Top 1 accuracy is 0.042
Top 2 accuracy is 0.069
Top 5 accuracy is 0.115
Top 10 accuracy is 0.161
Top 50 accuracy is 0.3
Top 100 accuracy is 0.371


In [9]:
getpred.get_prediction_on_sentence("आराम और सामग्री की स्थिति में।")

['glücklicherweise',
 'zurücklehnen',
 'losgelassen',
 'ruhig',
 'eingeschlafen',
 'stundenlang',
 'geschlafen',
 'entspannt',
 'frustrierend',
 'hektisch',
 'hinterher',
 'nachdenklich',
 'anstrengend',
 'aber',
 'Aber',
 'regungslos',
 'doch',
 'So',
 'so',
 'SO',
 'erfreulicherweise',
 'verkrampft',
 'trotzdem',
 'erfrischend',
 'aushalten',
 'gelangweilt',
 'verständlicherweise',
 'schlafen',
 'aufgeregt',
 'zumachen',
 'Zumachen',
 'ausruhen',
 'ehrlicherweise',
 'vorsichtig',
 'Bettdecke',
 'gewöhnt',
 'zurückhalten',
 'minutenlang',
 'erstaunlicherweise',
 'verschlafen',
 'Verschlafen',
 'augenblicklich',
 'unbequem',
 'ungeduldig',
 'ungehalten',
 'durchatmen',
 'gemütlich',
 'Beruhigen',
 'beruhigen',
 'drängeln']